# 配置信息

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

model_name_or_path = '/home/abc/mydisk/model/chatglm3-6b'  # 模型路径
peft_model_path = "/home/abc/mydisk/model/chatglm3-6b/out_zhouyi/checkpoint-200/"  # 微调模型路径
training_tag="ChatGLM3-6B(checkpoint-200, train_loss≈0.0006，数据集小，可能过拟合)" # 微调情况

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

config = PeftConfig.from_pretrained(peft_model_path)

In [3]:
# 加载模型，tokenizer
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True)

qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

base_model.requires_grad_(False)
base_model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=4608, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
           

In [4]:
! nvidia-smi

Wed Jul 10 23:02:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10                     Off |   00000000:00:08.0 Off |                    0 |
|  0%   43C    P0             58W /  150W |    4320MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# 比较方法，比较微调前后的结果
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

# 开始比较

In [6]:
base_response, ft_response = compare_chatglm_results("周易中包含哪些卦象", base_model, qlora_model, training_tag)

问题：周易中包含哪些卦象

原始输出：
在周易中，卦象是由两个卦象叠加而成，一个在上，一个在下，形成一种特定的卦象。一共有64卦，其中，坎卦、乾卦、坤卦被称为三大卦，其余的32卦都是通过叠加两个卦象而形成的。每一卦都有其特定的含义和哲学，涉及到天地人之间的关系。在周易中，卦象的解释和理解是非常重要的，它直接关系到对卦象的解读和应对。


微调后（ChatGLM3-6B(checkpoint-200, train_loss≈0.0006，数据集小，可能过拟合)）：
[gMASK] sop 周易中包含哪些卦象？ 在周易中，共有64卦，分为两个大卦：奇数卦和偶数卦。奇数卦包括奇数位数的卦象，如乾、奇数位的坤、奇数位的震、奇数位的巽、奇数位的坎、奇数位的离，以及奇数位的艮。偶数卦则包括偶数位数的卦象，如偶数位次的坤、偶数位次的震、偶数位次的巽、偶数位次的坎、偶数位次的离，以及偶数位次的艮。 

在周易中，每一卦都有其独特的含义和象征，每一卦也都有不同的名字和卦象。这些卦象和名字都包含了宇宙的哲学和人生智慧，对于解读人生和解决困难都有重要的指导意义。


In [7]:
base_response, ft_response = compare_chatglm_results("乾卦和坤卦一般代表什么", base_model, qlora_model, training_tag)

问题：乾卦和坤卦一般代表什么

原始输出：
在周易中，乾卦和坤卦是两个最基本的卦象，它们分别代表不同的卦义。

乾卦是由三个阳爻组成，象征着天，代表着正义、刚健和希望。它象征着天地的真理，强调刚健正义，希望获得吉祥。乾卦的卦象是天卦，代表着天地之德，以及君子观者之象。乾卦的卦象是一个卦象，由两个卦象叠加而成，上卦是乾，下卦是坤。乾卦代表天，坤卦代表地。

坤卦是由三个阴爻组成，象征着地，它代表刚健、正派和无私。坤卦象征着天地之厚，以及君子观者之象。坤卦的卦象是一个卦象，由两个卦象叠加而成，上卦是坤，下卦是坤。坤卦代表地，代表着刚健、正派和无私。

乾卦和坤卦在周易中具有很高的地位，它们分别代表不同的卦义，并且相互依存，相互制约。在解读乾卦和坤卦时，需要综合考虑，方能准确地理解其卦象所代表的意义。在个人成长和人生道路中，乾卦和坤卦分别代表着正义、刚健和正义、无私的品质，它们都是成长过程中必须拥有的品质。


微调后（ChatGLM3-6B(checkpoint-200, train_loss≈0.0006，数据集小，可能过拟合)）：
[gMASK] sop 乾卦和坤卦一般代表什么 乾卦和坤卦是周易中的两个卦象，分别由六个阳爻和六个阴爻组成。在周易中，卦象代表着宇宙的运行和变化，以及人类社会的现状和未来。乾卦和坤卦分别代表着天和地，象征着它们的不同特性和相互关系。

乾卦由六个阳爻组成，象征着天，象征着刚健、健行、刚健不屈的意境。乾卦代表天，象征刚健、健行，以及不屈不挠的精神。乾卦的核心哲学是：天行健，君子拟之。意思是要像天行健一样，坚强不屈，努力奋斗。乾卦的象征意义包括：刚健、健行、刚健不屈。

坤卦由六个阴爻组成，象征着地，象征着顺从、适应、包容的品德。坤卦代表地，象征着柔顺、和缓、包容，以及柔顺接受、适应环境的变化。坤卦的核心哲学是：地地相应，君子慎独。 meaning: 像地一样柔顺、包容，独自承担责任。坤卦的象征意义包括：和顺、和缓、包容，以及柔顺接受、适应环境的变化。

乾卦和坤卦在周易中相互对应，象征着天地相互依存，相互补足，形成一种和谐的关系。在个人成长中，乾卦和坤卦分别代表着刚健和柔顺的品质，象征着要刚柔并济，才能实现事业成功和人生幸福。在事业和经商中，乾卦和坤卦分别代表着刚健和柔顺的品质，象征着事业的发展和人际关系的处理需要刚柔并济，才能实现成功。在

In [8]:
base_response, ft_response = compare_chatglm_results("你知道水雷屯卦么", base_model, qlora_model, training_tag)

问题：你知道水雷屯卦么

原始输出：
水雷屯卦是一个由雷卦和屯卦组成的卦象，它代表着雷雷屯屯，即雷声在晴天中屯聚不散，预示着天气将雷雷屯聚，即将有雷雨天气。在这里，屯卦代表待，即等待，意味着君子应当等待时机的到来，然后采取行动。因此，这个卦象预示着君子应当耐心等待时机的到来，宜迟行。


微调后（ChatGLM3-6B(checkpoint-200, train_loss≈0.0006，数据集小，可能过拟合)）：
[gMASK] sop 你知道水雷屯卦么? 当然知道，水雷屯卦是一个由雷卦和坎卦组成的卦象，它象征着雷击坎，表示危险和困难。在这个卦象中，君子应当警惕并避免锋芒所指，从而避免灾难。对于经商者，必须小心谨慎，避免轻举妄动，灵活变通，才能化险为夷。


In [9]:
base_response, ft_response = compare_chatglm_results("你认为周易的卦象都合理么", base_model, qlora_model, training_tag)

问题：你认为周易的卦象都合理么

原始输出：
周易卦象是一种古老的占卜方式，它通过卦象的组合来揭示未来的吉凶。周易一卦分为两个部分，上卦和下卦，分别代表不同的卦象。通过卦象的组合，我们可以得出卦象的卦辞，从而得知这个卦象的吉祥或不吉祥。

在周易中，每个卦象都有其独特的含义和象征。比如，乾卦代表天，坤卦代表地，而其他卦象则分别代表不同的自然现象，如雷、风、雨等。通过卦象的组合，我们可以得出卦象的卦辞，从而得知这个卦象的吉祥或不吉祥。

虽然周易的卦象和卦辞都是古老的，但是它们仍然具有很高的参考价值。在现代，许多人会将周易的卦象和卦辞用于决策、预测和分析。然而，需要注意的是，周易的卦象和卦辞只是一种象征和参考，并不能完全决定未来的吉凶。因此，在运用周易卦象时，需要综合考虑卦象、卦辞、卦象的组合以及卦象的時运等因素。


微调后（ChatGLM3-6B(checkpoint-200, train_loss≈0.0006，数据集小，可能过拟合)）：
[gMASK] sop 你认为周易的卦象都合理么? 作为人工智能助手，我会根据我所学的知识来回答你的问题。
周易是一部古老的占卜卦象，它由八卦组合而成，一共有64卦。每一卦都有不同的含义和象征，它们分别代表了不同的象数、卦辞和卦象。周易的主要目的是预测事物的发展和变化，指导人们的行为。

虽然周易中的卦象和卦辞在古代被广泛应用于占卜和决策，但是随着时代的推移，人们对周易的理解和应用也产生了不同的观点和看法。有些人认为周易是迷信和伪科学，而另一些人则认为周易中蕴含着深刻的哲学和人生智慧。

无论如何，周易作为一部古老的占卜卦象，它反映了古人对自然和社会现象的观察和理解，也反映了古人的人生哲学和价值观。虽然现代社会已经不再像古代那样依赖占卜来预测未来，但周易仍然可以作为一种思考和决策的参考，帮助人们更好地理解自己和周围的世界。


In [10]:
! nvidia-smi

Wed Jul 10 23:12:52 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10                     Off |   00000000:00:08.0 Off |                    0 |
|  0%   45C    P0             59W /  150W |    4852MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----